# Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

## Шаг 1. Обзор данных

In [85]:
import pandas as pd
data = pd.read_csv('data.csv')
data.info()
data.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу


**Вывод**

В таблице 12 столбцов разных типов данных.
Согласно документации к данным:

* `children` — количество детей в семье
* `days_employed` — общий трудовой стаж в днях
* `dob_years` — возраст клиента в годах
* `education` — уровень образования клиента
* `education_id` — идентификатор уровня образования
* `family_status` — семейное положение
* `family_status_id` — идентификатор семейного положения
* `gender` — пол клиента
* `income_type` — тип занятости
* `debt` — имел ли задолженность по возврату кредитов
* `total_income` — ежемесячный доход
* `purpose` — цель получения кредита

В каждой строке таблицы - данные о клиентах банка. Колонки описывают характеристики клиента.

В данных встречаются:
* пропуски
* отрицательные значения
* дубликаты
* разный формат значений

## Шаг 2. Предобработка данных

### Обработка пропусков

In [86]:
# посчитаем пропуски в столбцах
data.isna().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

Пропуски встречаются в столбцах об общем трудовом стаже клиента и ежемесячном доходе. Такие пропуски возможно были допущены, так как не у всех клиентов есть официальная статистика зарплаты, не все могут разглашать уровень своего дохода, а так же  в данных есть пенсионеры, не получающие зарплату.

In [87]:
# Пропуски в столбце `days_employed` для нас по условиям задачи не важны, их можно заменить на нулевые значения.
data['days_employed'] = data['days_employed'].fillna(0)

Создадим фильтр для пропущенных значений.

In [88]:
data_filtered = data[data['total_income'].isna() == True]

Значения пропущены в столбце с ежемесячным уровнем дохода. 
Сгруппируем строки с пропущенными значениями в столбце `total_income` по типу занятости и подсчитаем количество клиентов для каждого типа.

In [89]:
data_filtered.groupby('income_type')['income_type'].count().sort_values(ascending=False)

income_type
сотрудник          1105
компаньон           508
пенсионер           413
госслужащий         147
предприниматель       1
Name: income_type, dtype: int64

Больше всего пропусков у сотрудников банка, далее у компаньонов, пенсионеров, госслужащих и предпринимателя. Найдем среднее значение ежемесячного дохода в каждой группе.

In [90]:
# сохраняем тип занятости в отдельный список
income_type = ['сотрудник', 'компаньон', 'пенсионер', 'госслужащий', 'предприниматель']

# создадим цикл, который сохраняет среднее значение по типу занятости в переменную и заполняет пропуски на это значение
for index in income_type:
    mean_total_income = data[data['income_type'] == index]['total_income'].mean()
    data.loc[(data['income_type'] == index)&(data['total_income'].isna() == True), 'total_income'] = mean_total_income
        
# проверим результат
data.isna().sum()

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

### Замена типа данных

In [91]:
# еще раз проверим тип данных в таблице
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     21525 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      21525 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


**Заменим вещественный тип данных на целочисленный**

В столбце `children` количество детей исправим с типа object на целочисленное значение int.
Приведем данные в столбцах `days_employed` и `total_income` к целочисленному значению с помощью метода `astype()`.

In [92]:
data['days_employed'] = data['days_employed'].astype('int')
data['total_income'] = data['total_income'].astype('int')
data['children'] = data['children'].astype('int')
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   children          21525 non-null  int32 
 1   days_employed     21525 non-null  int32 
 2   dob_years         21525 non-null  int64 
 3   education         21525 non-null  object
 4   education_id      21525 non-null  int64 
 5   family_status     21525 non-null  object
 6   family_status_id  21525 non-null  int64 
 7   gender            21525 non-null  object
 8   income_type       21525 non-null  object
 9   debt              21525 non-null  int64 
 10  total_income      21525 non-null  int32 
 11  purpose           21525 non-null  object
dtypes: int32(3), int64(4), object(5)
memory usage: 1.7+ MB


### Чистка таблицы

**Проверим столбцы на их необходимость для исследования.**

In [93]:
display (data.head(3))
print (data[data['education_id'] == 0]['education'].value_counts())
print (data[data['family_status_id'] == 1]['family_status'].value_counts())

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья
1,1,-4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля
2,0,-5623,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья


высшее    4718
ВЫСШЕЕ     274
Высшее     268
Name: education, dtype: int64
гражданский брак    4177
Name: family_status, dtype: int64


Колонки `education_id` и `family_status_id` содержат численное обозначение типа образования и семейного статуса, данные колонки нам не пригодятся, в ином случае, мы можем их восстановить, заменив тип численным значением.

In [94]:
data = data.drop('education_id', axis=1)
data = data.drop('family_status_id', axis=1)
data.head(5)

,children,days_employed,dob_years,education,family_status,gender,income_type,debt,total_income,purpose
0,1,-8437,42,высшее,женат / замужем,F,сотрудник,0,253875,покупка жилья
1,1,-4024,36,среднее,женат / замужем,F,сотрудник,0,112080,приобретение автомобиля
2,0,-5623,33,Среднее,женат / замужем,M,сотрудник,0,145885,покупка жилья
3,3,-4124,32,среднее,женат / замужем,M,сотрудник,0,267628,дополнительное образование
4,0,340266,53,среднее,гражданский брак,F,пенсионер,0,158616,сыграть свадьбу


**Проверим аномальные значения.**

In [95]:
print (data['children'].value_counts())
print (data['gender'].value_counts())
print (data['debt'].value_counts())

 0     14149
 1      4818
 2      2055
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64
F      14236
M       7288
XNA        1
Name: gender, dtype: int64
0    19784
1     1741
Name: debt, dtype: int64


Предположительно: в столбце количества детей в строках со значениями 20 и -1 клиент ошибочно указал количество детей и значение соответственно равно 2 и 1. 

В столбце пола указано 1 аномальное значение, можем его рандомно заменить на м/ж.

In [96]:
# исправим аномальные значения
data.loc[(data['children'] == 20), 'children'] = 2
data.loc[(data['children'] == -1), 'children'] = 1

# проверим результат
data['children'].value_counts()

0    14149
1     4865
2     2131
3      330
4       41
5        9
Name: children, dtype: int64

In [97]:
data.loc[(data['gender'] == 'XNA'), 'gender'] = 'M'
print (data['gender'].value_counts())

F    14236
M     7289
Name: gender, dtype: int64


### Обработка дубликатов

**Избавимся от неявных дубликатов.**

In [98]:
# посчитаем количество уникальных значений в столбце `education`
data['education'].unique()

array(['высшее', 'среднее', 'Среднее', 'СРЕДНЕЕ', 'ВЫСШЕЕ',
       'неоконченное высшее', 'начальное', 'Высшее',
       'НЕОКОНЧЕННОЕ ВЫСШЕЕ', 'Неоконченное высшее', 'НАЧАЛЬНОЕ',
       'Начальное', 'Ученая степень', 'УЧЕНАЯ СТЕПЕНЬ', 'ученая степень'],
      dtype=object)

Так как значения в столбце отличаются только верхним регистром, приведем все значения к нижнему регистру методом `str.lower()`.

In [99]:
data['education'] = data['education'].str.lower()
# проверим
data['education'].unique()

array(['высшее', 'среднее', 'неоконченное высшее', 'начальное',
       'ученая степень'], dtype=object)

**Посчитаем количество явных дубликатов.**

In [100]:
data.duplicated().sum()

71

In [101]:
# удаляем явные дубликаты
data = data.drop_duplicates().reset_index(drop=True)
display(data.duplicated().sum())

0

Мы избавились от явных и неявных дубликатов в данных.
Привели столбец образования к единому стилю.


### Лемматизация

Выведем на экран список уникальных названий целей получения кредита, отсортированный в алфавитном порядке.

In [102]:
data['purpose'].unique()

array(['покупка жилья', 'приобретение автомобиля',
       'дополнительное образование', 'сыграть свадьбу',
       'операции с жильем', 'образование', 'на проведение свадьбы',
       'покупка жилья для семьи', 'покупка недвижимости',
       'покупка коммерческой недвижимости', 'покупка жилой недвижимости',
       'строительство собственной недвижимости', 'недвижимость',
       'строительство недвижимости', 'на покупку подержанного автомобиля',
       'на покупку своего автомобиля',
       'операции с коммерческой недвижимостью',
       'строительство жилой недвижимости', 'жилье',
       'операции со своей недвижимостью', 'автомобили',
       'заняться образованием', 'сделка с подержанным автомобилем',
       'получение образования', 'автомобиль', 'свадьба',
       'получение дополнительного образования', 'покупка своего жилья',
       'операции с недвижимостью', 'получение высшего образования',
       'свой автомобиль', 'сделка с автомобилем',
       'профильное образование', 'высшее об

In [119]:
from nltk.stem.snowball import SnowballStemmer
stemmer = SnowballStemmer("russian") 

In [120]:
# создадим функцию для замены на лемматизированные значения в столбце purpose

def create_category_purpose(row):
    
    l = stemmer.stem(row['purpose'])
    
    if ('автомобиль' in l) or ('автомоб' in l ) or ('автомобил' in l ) :
        return 'покупка автомобиля'
    
    if ('образование' in l) or ('образован' in l ) :
        return 'получение образования'
    
    if (('жилье' in l) or ('недвижимость' in l ) or ('недвижим' in l ) or ('жилья ' in l )
        or ('жил ' in l ) or ('жильем ' in l ) or ('недвижимости ' in l )):
        return 'покупка недвижимости'
    
    if ('свадьба' in l) or ('свадьб' in l ) or ('свадьбу' in l ):
        return 'на свадьбу'
    
    else:
        return row['purpose']

In [121]:

data['purpose'] = data.apply(create_category_purpose, axis=1)


In [122]:
# проверим выведя список уникальных значений в столбце
display (data['purpose'].unique())
data.head()

array(['покупка жилья', 'покупка автомобиля', 'получение образования',
       'на свадьбу', 'операции с жильем', 'покупка недвижимости', 'жилье',
       'покупка своего жилья', 'ремонт жилью'], dtype=object)

,children,days_employed,dob_years,education,family_status,gender,income_type,debt,total_income,purpose
0,1,-8437,42,высшее,женат / замужем,F,сотрудник,0,253875,покупка жилья
1,1,-4024,36,среднее,женат / замужем,F,сотрудник,0,112080,покупка автомобиля
2,0,-5623,33,среднее,женат / замужем,M,сотрудник,0,145885,покупка жилья
3,3,-4124,32,среднее,женат / замужем,M,сотрудник,0,267628,получение образования
4,0,340266,53,среднее,гражданский брак,F,пенсионер,0,158616,на свадьбу


Теперь значения в столбце `purpose` имеют 4 категории и готовы к следующему шагу исследования.

### Категоризация данных

In [123]:
# проверим уникальные значение в столбце children
data['children'].value_counts()

0    14091
1     4855
2     2128
3      330
4       41
5        9
Name: children, dtype: int64

Для распределения клиентов на категории по количеству детей создадим функцию.

In [124]:
def children_category (row):
    
    if row['children'] == 0:
        return 'детей нет'
    if row['children'] >= 3:
        return 'многодетные семьи'
    else:
        return 'дети есть'
    
# для проведения анализа нам достаточно данных о наличии или отсутствии детей, 
# поэтому разделим столбец `children` на 2 категории

data['children'] = data.apply(children_category, axis=1)
data.head(3)

,children,days_employed,dob_years,education,family_status,gender,income_type,debt,total_income,purpose
0,дети есть,-8437,42,высшее,женат / замужем,F,сотрудник,0,253875,покупка жилья
1,дети есть,-4024,36,среднее,женат / замужем,F,сотрудник,0,112080,покупка автомобиля
2,детей нет,-5623,33,среднее,женат / замужем,M,сотрудник,0,145885,покупка жилья


Для определения категорий клиентов по уровеню ежемесячного дохода,
находим минимальное, максимальное и медианное значение заработной платы.

In [125]:
display (data['total_income'].max())
display (data['total_income'].min())
display (data['total_income'].median())
display (data['total_income'].mean())

2265604

20667

151887.0

167431.58371399273

Медианная и средняя зарплата примерно одинаковы, 150000 - средний доход клиентов.

In [126]:
# находим верхнюю границу нижнего уровня доходов
below_average = data[data['total_income'] < 150000]
below_average['total_income'].median() 

106784.0

In [127]:
# находим нижнюю границу высокого уровня доходов
below_average = data[data['total_income'] > 150000]
below_average['total_income'].median()

201815.0

Таким образом, получается:

* `<` `100000` - низкий уровень дохода
* `<` `150000` - доходы ниже среднего
* `>` `150000` - доходы выше среднего
* `>` `200000` - высокий доход

Для разделения клиентов на категории создадим функцию.

In [128]:
def total_income_category (row):
    
    if row['total_income'] <= 100000:
        return 'низкий уровень дохода'
    if row['total_income'] < 150000 and row['total_income'] > 100000:
        return 'доход ниже среднего'
    if row['total_income'] >= 150000 and row['total_income'] < 200000:
        return 'доход выше среднего'
    if row['total_income'] >= 200000:
        return 'высокий уровень дохода'

# создадим новый столбец с категориями ежемесячного дохода
data['total_income_category'] = data.apply(total_income_category, axis=1)
data.head(5)

,children,days_employed,dob_years,education,family_status,gender,income_type,debt,total_income,purpose,total_income_category
0,дети есть,-8437,42,высшее,женат / замужем,F,сотрудник,0,253875,покупка жилья,высокий уровень дохода
1,дети есть,-4024,36,среднее,женат / замужем,F,сотрудник,0,112080,покупка автомобиля,доход ниже среднего
2,детей нет,-5623,33,среднее,женат / замужем,M,сотрудник,0,145885,покупка жилья,доход ниже среднего
3,многодетные семьи,-4124,32,среднее,женат / замужем,M,сотрудник,0,267628,получение образования,высокий уровень дохода
4,детей нет,340266,53,среднее,гражданский брак,F,пенсионер,0,158616,на свадьбу,доход выше среднего


**Вывод**

Мы обработали данные для исследования:
* обработали пропуски в признаках трудового стажа и дохода
* заменили вещественный тип данных на численный в столбцах трудового стажа, дохода и количества детей
* очистили таблицу от ненужных признаков
* обработали аномальные значения в столбцах количества детей и пола
* обработали дубликаты в столбце образования
* заменили значения в столбце цели для получения кредита
* категоризировали столбец с количеством детей, и поделили на 2 категории, так как по условиям задачи необходимо ответить на вопрос: "Есть ли зависимость между наличием детей и возвратом кредита в срок?"
* поделили столбец с уровнем дохода на 4 категории по медианному значению

## Шаг 3. Проверка гипотез

**Есть ли зависимость между наличием детей и возвратом кредита в срок?**

In [129]:
# создадим функцию для проверки гипотезы
def debt_status (row):
    
    if row['debt'] == 0:
        return 'нет задолженности'
    else:
        return 'есть задолженность'

data['debt_status'] = data.apply(debt_status, axis=1)

In [130]:
children_table = data.pivot_table(index = 'debt_status', columns = 'children', values = 'debt', aggfunc = 'count')
children_table.loc['доля задолженности'] = (children_table.iloc[0] / children_table.iloc[1]).map('{:.0%}'.format)
children_table

children,детей нет,дети есть,многодетные семьи
debt_status,,,
есть задолженность,1063,647,31
нет задолженности,13028,6336,349
доля задолженности,8%,10%,9%


**Вывод**

Отсутствие детей положительно влияет на вероятность возврата кредита в срок, так как доля задолженности у клиентов с 1, 2 детьми выше, чем у клиентов без детей.

**Есть ли зависимость между семейным положением и возвратом кредита в срок?**

In [131]:
family_table = data.pivot_table(index = 'debt_status', columns = 'family_status', values = 'debt', aggfunc = 'count')
family_table.loc['доля задолженности'] = (family_table.iloc[0] / family_table.iloc[1]).map('{:.0%}'.format)
family_table

family_status,Не женат / не замужем,в разводе,вдовец / вдова,гражданский брак,женат / замужем
debt_status,,,,,
есть задолженность,274,85,63,388,931
нет задолженности,2536,1110,896,3763,11408
доля задолженности,11%,8%,7%,10%,8%


**Вывод**

Брак положительно влияет на вероятность возврата кредита в срок. Так же статус "в разводе" и "вдовец/вдова" имеют меньшую долю должников, чем у людей со статусом "не женат/не замужем" или проживающих в гражданском браке.

**Есть ли зависимость между уровнем дохода и возвратом кредита в срок?**

In [132]:
finance_table = data.pivot_table(index = 'debt_status', columns = 'total_income_category', values = 'debt', aggfunc = 'count')
finance_table.loc['доля задолженности'] = (finance_table.iloc[0] / finance_table.iloc[1]).map('{:.0%}'.format)
finance_table

total_income_category,высокий уровень дохода,доход выше среднего,доход ниже среднего,низкий уровень дохода
debt_status,,,,
есть задолженность,388,473,526,354
нет задолженности,5180,4860,5564,4109
доля задолженности,7%,10%,9%,9%


**Вывод**

* Клиенты с высоким уровнем дохода (>200000) наиболее надежны, и с большей вероятностью вернут кредит в срок. 
* Клиенты с низким доходом и доходом ниже среднего имеют чуть больший процент задолженности и менее надежны. 
* Клиенты с доходом выше среднего с меньшей вероятностью вернут кредит в срок.

**Как разные цели кредита влияют на его возврат в срок?**

In [133]:
purpose_table = data.pivot_table(index = 'debt_status', columns = 'purpose', values = 'debt', aggfunc = 'count')
purpose_table.loc['доля задолженности'] = (purpose_table.iloc[0] / purpose_table.iloc[1]).map('{:.0%}'.format)
purpose_table

purpose,жилье,на свадьбу,операции с жильем,покупка автомобиля,покупка жилья,покупка недвижимости,покупка своего жилья,получение образования,ремонт жилью
debt_status,,,,,,,,,
есть задолженность,46,186,48,403,48,571,34,370,35
нет задолженности,600,2138,604,3903,598,7069,586,3643,572
доля задолженности,8%,9%,8%,10%,8%,8%,6%,10%,6%


**Вывод**

* Кредит на покупку автомобиля или получение образования с меньшей вероятностью будет возвращен в срок
* Клиент, который оформляет кредит на покупку недвижимости с большей вероятностью вернет его в срок.

## Шаг 4. Общий вывод

* На возврат кредита в срок влияют все 4 критерия: наличие детей, семейный статус, уровень дохода и цель получения кредита.
* В большей степени на возврат кредита в срок влияет семейное положение, так как разница между наименьшей (7%) и наибольшей (11%) долями - 4%.
* Наиболее надежный клиент банка - вдовец/вдова, без детей, с высоким уровнем дохода, который берет кредит на недвижимость.
* Наименее надежный клиент банка - не женат/не замужем, с детьми, с доходом выше среднего, который берет кредит на покупку автомобиля или получение образования.